In [ ]:
import pandas as pd
data = pd.read_csv('../data_train/stringtie_tss_labeled.csv')
print(data['label'].value_counts())
# print min and max delta coverage rows
print(data.loc[data['delta_coverage'].idxmax()])
print(data.loc[data['delta_coverage'].idxmin()])
# print(data['delta_coverage'].min())

label
0    6851
1    5639
Name: count, dtype: int64
12134
10607
